In [1]:
import pandas as pd
import datetime


n_his = 24
n_pred = 12

col = pd.read_csv('./dataset/col_ap.csv', header=None).values.flatten()

specific_date = datetime.datetime(2021, 5, 11)

idx = pd.date_range(start=(specific_date), end=(specific_date + datetime.timedelta(days=1)), freq='5min').values[(n_his + n_pred -1):-1]
idx = pd.DatetimeIndex(data=idx, tz='Etc/GMT+2')

week_num = datetime.date.today().weekday() # week_num = 0 is Monday, and 6 is Sunday.
seasonal = pd.read_csv('./dataset/seasonal_week.csv', header=None).values[(week_num * 288): ((week_num +1)* 288)][(n_his + n_pred -1):]
df_seasonal = pd.DataFrame(seasonal, index=idx, columns=col)

csv_file_path = './save/seasonal.csv'
df_seasonal.to_csv(csv_file_path, index=False)



In [2]:
from load_data import *
import pandas as pd
import torch
from stgcn import STGCN
from utils import *
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler

matrix_path = "dataset/03062021_M_96.csv"

n_his = 24
n_pred = 12
n_route = 96
Ks, Kt = 3, 3
blocks = [[1, 32, 64], [64, 32, 128]]
drop_prob = 0

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

W = load_matrix(matrix_path)
L = scaled_laplacian(W)
Lk = cheb_poly(L, Ks)
Lk = torch.Tensor(Lk.astype(np.float32)).to(device)


n_his = 24
n_pred = 12
day_slot = 288

In [6]:
data = pd.read_csv("./dataset/03062021_Pred_adj_96.csv", header=None).values

mean_values = data.mean()
std_values = data.std()

scaler = StandardScaler()

data = scaler.fit_transform(data)

print(mean_values)
print(std_values)


x_ , y_  = data_transform(data, n_his, n_pred, day_slot, device)


model_state_dict = torch.load('./save/model.pt' , map_location=torch.device('cpu'))

model = STGCN(Ks, Kt, blocks, n_his, n_route, Lk, drop_prob)  # Assuming your STGCN model class is named STGCN
model.load_state_dict(model_state_dict)

# Step 4: Perform inference
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    predicted_outputs = model(x_)

reshaped_pred = predicted_outputs.reshape(253, 96)
numpy_array = reshaped_pred.numpy()


# inverse normalization on the input data
numpy_array = numpy_array * std_values + mean_values

# import column names
col_ap = pd.read_csv("./dataset/col_ap.csv", header=None).values.flatten()

# Step 2: Create timestamps
num_timesteps = numpy_array.shape[0]  # Assuming the number of timesteps is the first dimension of numpy_array
time_diff = datetime.timedelta(minutes=5)  # Assuming each timestep is 5 minutes
# specific_date = datetime.datetime(2021, 5, 11)  # Specific date for the first timestamp
specific_date = datetime.datetime(2021, 5, 11, 2, 55, tzinfo=datetime.timezone(datetime.timedelta(hours=-2)))

timestamps = [specific_date + i * time_diff for i in range(num_timesteps)]

# Step 3: Combine timestamps and predicted data into a DataFrame
data = np.concatenate((np.array(timestamps).reshape(-1, 1), numpy_array), axis=1)

# Create a DataFrame
# df_adj = pd.DataFrame(data, columns=['timestamp'] + col_ap.tolist())
df_adj = pd.DataFrame(numpy_array, index=idx, columns=col)

col = pd.read_csv('./dataset/col_ap.csv', header=None).values.flatten()

specific_date = datetime.datetime(2021, 5, 11)

idx = pd.date_range(start=(specific_date), end=(specific_date + datetime.timedelta(days=1)), freq='5min').values[(n_his + n_pred -1):-1]
idx = pd.DatetimeIndex(data=idx, tz='Etc/GMT+2')

# week_num = datetime.date.today().weekday() # week_num = 0 is Monday, and 6 is Sunday.
week_num = 2
seasonal = pd.read_csv('./dataset/seasonal_week.csv', header=None).values[(week_num * 288): ((week_num +1)* 288)][(n_his + n_pred -1):]
df_seasonal = pd.DataFrame(seasonal, index=idx, columns=col)
# df_adj = pd.DataFrame(data, index=idx, columns=col)

df = df_adj + df_seasonal 

# df["Timesteps"]  = timestamps
df.insert(0, 'Timesteps', timestamps)


# Save the DataFrame to a CSV file
csv_file_path = './save/9_april_pred.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at '{csv_file_path}'")

7.681348929506656
9.48626654475964


RuntimeError: Error(s) in loading state_dict for STGCN:
	size mismatch for st_conv1.tconv2.conv.weight: copying a param with shape torch.Size([128, 32, 3, 1]) from checkpoint, the shape in current model is torch.Size([64, 32, 3, 1]).
	size mismatch for st_conv1.tconv2.conv.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for st_conv2.tconv2.conv.weight: copying a param with shape torch.Size([256, 32, 3, 1]) from checkpoint, the shape in current model is torch.Size([128, 32, 3, 1]).
	size mismatch for st_conv2.tconv2.conv.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).

In [14]:
df = pd.read_csv('./dataset/03062021_Pred_adj_96.csv')
df_k = pd.read_csv('./dataset/03062021_Pred_adj_96_k.csv')

numy_array = df_k.to_numpy()


# numpy_array = numpy_array * std_values + mean_values

# print(std)
# print(mean)

# csv_file_path = './save/output.csv'
# np.savetxt(csv_file_path, numpy_array, delimiter=',')
    
# print(predicted_outputs.size())    
print(numpy_array.shape)
# print(y_)

col_ap = pd.read_csv("./dataset/col_ap.csv", header=None).values.flatten()

# Step 2: Create timestamps
num_timesteps = numpy_array.shape[0]  # Assuming the number of timesteps is the first dimension of numpy_array
time_diff = datetime.timedelta(minutes=5)  # Assuming each timestep is 5 minutes
# specific_date = datetime.datetime(2021, 5, 11)  # Specific date for the first timestamp
specific_date = datetime.datetime(2021, 5, 11, 2, 55, tzinfo=datetime.timezone(datetime.timedelta(hours=-2)))

timestamps = [specific_date + i * time_diff for i in range(num_timesteps)]

# Step 3: Combine timestamps and predicted data into a DataFrame
data = np.concatenate((np.array(timestamps).reshape(-1, 1), numpy_array), axis=1)

# Create a DataFrame
# df_adj = pd.DataFrame(data, columns=['timestamp'] + col_ap.tolist())
df_adj = pd.DataFrame(numpy_array, index=idx, columns=col)

col = pd.read_csv('./dataset/col_ap.csv', header=None).values.flatten()

specific_date = datetime.datetime(2021, 5, 11)

idx = pd.date_range(start=(specific_date), end=(specific_date + datetime.timedelta(days=1)), freq='5min').values[(n_his + n_pred -1):-1]
idx = pd.DatetimeIndex(data=idx, tz='Etc/GMT+2')

# week_num = datetime.date.today().weekday() # week_num = 0 is Monday, and 6 is Sunday.
week_num = 1
seasonal = pd.read_csv('./dataset/seasonal_week.csv', header=None).values[(week_num * 288): ((week_num +1)* 288)][(n_his + n_pred -1):]
df_seasonal = pd.DataFrame(seasonal, index=idx, columns=col)
# df_adj = pd.DataFrame(data, index=idx, columns=col)

df = df_adj + df_seasonal 

# df["Timesteps"]  = timestamps
df.insert(0, 'Timesteps', timestamps)


# Save the DataFrame to a CSV file
csv_file_path = './save/prediction_data_kent.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at '{csv_file_path}'")

(253, 96)
CSV file saved at './save/prediction_data_kent.csv'
